In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import torch
from torchvision import datasets, transforms
import numpy as np
import pandas as pd
from dataloader import ImageDataSet

In [2]:
trainset = ImageDataSet(pd.read_csv('Data/RankedPairs.csv').drop(['SortKey'],axis=1),64,torch.device('cpu'))
trainloader = torch.utils.data.DataLoader(trainset,batch_size=6,num_workers=0,shuffle=False,pin_memory=True)
dataset = ImageDataSet(pd.read_csv('Data/ImageList.csv'),64,torch.device('cpu'))
dataloader = torch.utils.data.DataLoader(dataset,batch_size=100,num_workers=0,shuffle=False,pin_memory=True)

In [3]:
train_csv = pd.read_csv('Data/RankedPairs.csv')
sort_key = [x for x in range(len(trainset))]
sort_key = np.reshape(sort_key,(-1,2))
np.random.shuffle(sort_key)
sort_key = np.reshape(sort_key,(-1))

In [4]:
train_csv

,ImageFile,Rating,SortKey
0,OIP-tr0q1fDCT9CpzzLXvKzplQHaFj.jpeg,0,0
1,OIP-K2IJLZ51b7jxUZ5BqbK8tQHaFj.jpeg,1,1
2,OIP-PVC0OpGJYKWGPXgR9YqYQgHaE8.jpeg,0,2
3,OIP-K2IJLZ51b7jxUZ5BqbK8tQHaFj.jpeg,1,3
4,OIP-fkPYxdnaJeLEg8SbGLRdEgDVEl.jpeg,0,4
...,...,...,...
817,OIP-khE9whYza1vquFILWCebZAHaE8.jpeg,1,817
818,OIP-DhTPBAA73c9nBoMKHeumiQHaE7.jpeg,0,818
819,OIP-DkG6w7oaGzMOQhSpdEJx9wHaKo.jpeg,1,819
820,OIP-A5A5NAm6LQ0MIiukjJlDAAHaEK.jpeg,0,820


In [5]:
train_csv['SortKey'] = sort_key
train_csv.sort_values(by=['SortKey'],inplace=True,ascending=True)
train_csv.reset_index(drop=True,inplace=True)
train_csv

,ImageFile,Rating,SortKey
0,136.jpeg,0,0
1,137.jpeg,1,1
2,OIP-xG0w41Y3RCTk8EuFgU2pBAHaFG.jpeg,0,2
3,OIP-K2IJLZ51b7jxUZ5BqbK8tQHaFj.jpeg,1,3
4,176.jpeg,1,4
...,...,...,...
817,OIP-iitQsbjzA89x0-EKXWExAQHaEK.jpeg,1,817
818,OIP-i467eIS30UhA3i4xKFmY-wHaFk.jpeg,0,818
819,OIP-i787frQ771CACCzrPtDeVAHaF7.jpeg,1,819
820,112.jpeg,1,820


In [12]:
imagelist = iter(trainloader)
images = next(imagelist)
fig = plt.figure(figsize = (10,10))
n_images = 12
for n in n_images:
ax1 = fig.add_subplot(321)
ax1.imshow(np.transpose(images[0][0], (1,2,0)), interpolation='nearest')
ax2 = fig.add_subplot(322)
ax2.imshow(np.transpose(images[0][1], (1,2,0)), interpolation='nearest')
ax3 = fig.add_subplot(323)
ax3.imshow(np.transpose(images[0][2], (1,2,0)), interpolation='nearest')
ax4 = fig.add_subplot(324)
ax4.imshow(np.transpose(images[0][3], (1,2,0)), interpolation='nearest')
ax5 = fig.add_subplot(325)
ax5.imshow(np.transpose(images[0][4], (1,2,0)), interpolation='nearest')
ax6 = fig.add_subplot(326)
ax6.imshow(np.transpose(images[0][5], (1,2,0)), interpolation='nearest')
fig.show()

IndentationError: expected an indented block (<ipython-input-12-48baa3f53353>, line 6)

In [26]:
def test(model, iterator):
    test_result = []
    model.eval()
    with torch.no_grad():
        for batch in iterator:
            images = batch[0]
            predictions = torch.flatten(model(images))
            test_result.extend(predictions.tolist())
    return test_result

In [27]:
from model import CNNSingleValueRanker
#Define the net characteristics

n_initial_channels = 3
n_conv_blocks = 5
image_size = 64

#Define the training characteristics
train_batch_size = 20
full_set_rank_batch_size = 100

model = CNNSingleValueRanker(image_size,n_initial_channels,n_conv_blocks)
try:
    model.load_state_dict(torch.load('RankPrediction-model.pkl'))
except:
    print('no previously existing trained model')
model = model.to(torch.device('cpu'))

In [28]:
dataset = ImageDataSet(pd.read_csv('Data/ImageList.csv'),64,torch.device('cpu'))
dataloader = torch.utils.data.DataLoader(dataset,batch_size=100,num_workers=0,shuffle=False,pin_memory=True)
predictions = test(model,dataloader)
test_result_df = pd.read_csv('Data/ImageList.csv')
test_result_df['Rating']=predictions
test_result_df.sort_values(by=['Rating'],inplace=True,ascending=False)
test_result_df.reset_index(drop=True)
test_result_df.to_csv('Data/ImageList.csv',index=False)


,ImageFile,Rating
0,OIP-qRVEhxhzA2Ph71AVxy95GwHaEo.jpeg,2.606380
1,1713.jpeg,2.493047
2,OIP-aHyHD9XrtbUVeY0pZaWT3gHaFj.jpeg,2.176726
3,OIP-FUcOvcfngNRNfZ1s7o7xmAHaFj.jpeg,2.161285
4,OIP-wcqKiFBuN3jMTuOc_rphygHaHa.jpeg,2.160398
...,...,...
25835,OIP-omIV2JNmN7um8lv0SFED1gHaFo.jpeg,-5.047529
25836,OIP-EJZk0GSdY5PG5FvxJpRjDAHaFA.jpeg,-5.133920
25837,OIP-EgQrXtinWal9y2l9rlR_MgHaE8.jpeg,-5.444434
25838,OIP-P2tv0XM_koL73268qeIhMgHaE8.jpeg,-5.508602
